# Import Libary

In [ ]:
# Install libary necessary
!pip install -Uqqq pip --progress-bar off
!pip install -qqq torch==2.1.0 --progress-bar off
!pip install -qqq transformers==4.34.0 --progress-bar off
!pip install -qqq langchain --progress-bar off
!pip install -qqq chromadb --progress-bar off
!pip install -qqq xformers==0.0.20 --progress-bar off
!pip install -qqq sentence_transformers==2.2.2 --progress-bar off
!pip install -qqq InstructorEmbedding==1.0.1 --progress-bar off
!pip install -qqq websocket-client --progress-bar off
!pip install -qqq gradio --progress-bar off
!pip install -qqq python-engineio==3.14.2 python-socketio==4.6.0 --progress-bar off
!wget -q https://github.com/PanQiWei/AutoGPTQ/releases/download/v0.4.1/auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl
!pip install -qqq auto_gptq-0.4.1+cu118-cp310-cp310-linux_x86_64.whl --progress-bar off
!sudo apt-get install poppler-utils

In [ ]:
import torch
from auto_gptq import AutoGPTQForCausalLM
from langchain import HuggingFacePipeline, PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader, DirectoryLoader, TextLoader
from langchain.vectorstores import Chroma
from transformers import AutoTokenizer, TextStreamer, pipeline
import locale

import re
import requests
import json
import time
from datetime import datetime

DEVICE = "cuda:0" if torch.cuda.is_available() else "cpu"


In [ ]:
# Connect to drive
from google.colab import drive
drive.mount('/content/drive')

# Data
- Data gathering and preprocessing


In [ ]:
import json
with open('/content/drive/MyDrive/demo_websocket/list_stock.json', 'r') as f:
      data_stocks = json.load(f)
symbols = []  # Add more symbols if needed

for i in range(0,len(data_stocks['results'])):
  symbols.append(data_stocks['results'][i]['symbol'])

In [ ]:
exchange_slugs = []  # Add more exchanges if needed
symbols = []  # Add more symbols if needed

for i in range(0,len(data_stocks['results'])):
  symbols.append(data_stocks['results'][i]['symbol'])
  exchange_slugs.append(data_stocks['results'][i]['exchange_slug'])

In [ ]:
import requests

def financial_data(url, exchange_slug, symbol):
    data = {
        'exchange_slug': exchange_slug,
        'symbol': symbol
    }

    response = requests.post(url, json=data)
    if response.status_code == 200:
        try:
            result = response.json()
            return result
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON response from URL {url}: {e}")
            return None
    else:
        print(f"Yêu cầu không thành công cho URL {url}. Mã trạng thái: {response.status_code}")
        return None

urls = [
    your url US Stock API]



result_filenames = {
    urls[0]: "Revenue.txt",
    urls[1]: "Net-Income.txt",
    urls[2]: "Cash-Flow.txt",
    urls[3]: "EPS.txt",
    urls[4]: "DTER.txt"
}
financials_labels = ["Revenue", "Net Income", "Cash Flow", "EPS", "DTER"]

In [ ]:
from datetime import datetime
import math

for url in urls:
    for i in range(len(symbols)):
        result = financial_data(url, exchange_slugs[i], symbols[i])
        if result is not None:
            filename = result_filenames.get(url, "Unknown.txt")
            label_index = urls.index(url)

            with open(filename, 'a') as txt_file:
                for index, entry in enumerate(result['data']):
                    value = entry['value']
                    time = entry['time']
                    dt = datetime.now()
                    txt_file.write(f"{financials_labels[label_index]} of {symbols[i]} stock is {value} on {time}\n")
                    txt_file.write(f"On {time},{financials_labels[label_index]} of {symbols[i]} stock is {value}\n")

                    year = (datetime.strptime(time, "%Y-%m-%d")).year
                    month = str(math.ceil(int((datetime.strptime(time, "%Y-%m-%d")).month)/3))
                    month_recent=str(math.floor(dt.month/3))
                    if month_recent=='0':
                       month_recent='4'
                    txt_file.write(f"{financials_labels[label_index]} of {symbols[i]} stock is {value} in the Q{month} of {year}\n")
                    txt_file.write(f"In the Q{month} of {year},{financials_labels[label_index]} of {symbols[i]} stock is {value}\n")

                    if f'Q{month}' == "Q1":
                      txt_file.write(f"{financials_labels[label_index]} of {symbols[i]} stock is {value} in the first quarter of {year}\n")
                    elif f'Q{month}' == "Q2":
                      txt_file.write(f"{financials_labels[label_index]} of {symbols[i]} stock is {value} in the second quarter of {year}\n")
                    elif f'Q{month}' == "Q3":
                      txt_file.write(f"{financials_labels[label_index]} of {symbols[i]} stock is {value} in the third quarter of {year}\n")
                    elif f'Q{month}' == "Q4":
                      txt_file.write(f"{financials_labels[label_index]} of {symbols[i]} stock is {value} in the fourth quarter of {year}\n")
                    if month_recent == month and dt.year == year:
                      txt_file.write(f"{financials_labels[label_index]} of {symbols[i]} stock is {value} in the last quarter of {year}\n")
                      txt_file.write(f"{financials_labels[label_index]} of {symbols[i]} stock now is {value}\n")
                      txt_file.write(f"{financials_labels[label_index]} of {symbols[i]} stock recent is {value}\n")

In [ ]:

# Define a list of filenames to merge
filenames_to_merge = ["Revenue.txt", "Net-Income.txt", "Cash-Flow.txt", "EPS.txt", "DTER.txt"]

# Name of the output merged file
output_filename = "/content/drive/MyDrive/data/financial_data.txt"

# Open the output file in write mode to overwrite existing data
with open(output_filename, 'w') as output_file:
    # Loop through each filename to merge
    for filename in filenames_to_merge:
        try:
            with open(filename, 'r') as input_file:
                # Read the content of the input file
                content = input_file.read()
                # Append the content to the output file
                output_file.write(content)
                # Add a separator between merged files
                output_file.write(f"--\n")
            print(f"File {filename} has been merged into {output_filename}")
        except FileNotFoundError:
            print(f"File {filename} not found, skipping.")

print(f"All files have been merged into {output_filename} and existing data has been overwritten.")


# Functions
- load data
- split text
- model embedding
- model LLM

In [ ]:
# Load Data from drive
def load_documents():
    loader = DirectoryLoader('/content/drive/MyDrive/Data', glob="*.txt", loader_cls=TextLoader)
    documents = loader.load()
    return documents


def split_text_into_chunks(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=70, chunk_overlap=0)
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

# Load model embeddings ember-v1
def create_embeddings():
    embeddings = HuggingFaceInstructEmbeddings(
      model_name="llmrails/ember-v1", model_kwargs={"device": DEVICE}
    )
    return embeddings

# Load Chroma DB and store vector embeddings
def create_vector_store(text_chunks, embeddings):
    db = Chroma.from_documents(text_chunks, embeddings, persist_directory="db")
    return db

# Load model LLMs
def create_llms_model():
    model_name_or_path = "TheBloke/Llama-2-13b-Chat-GPTQ"
    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
    model = AutoGPTQForCausalLM.from_quantized(
        model_name_or_path,
        revision="gptq-4bit-32g-actorder_True",
        # model_basename=model_basename,
        use_safetensors=True,
        trust_remote_code=True,
        inject_fused_attention=False,
        device=DEVICE,
        quantize_config=None,
    )
    return model, tokenizer


In [ ]:
embeddings = create_embeddings()

In [ ]:
model, tokenizer = create_llms_model()

# Define System Prompt

In [ ]:
# Format output use Prompt
DEFAULT_SYSTEM_PROMPT = """
You are a helpful, respectful and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature.

If a question does not make any sense, or is not factually coherent, explain why instead of answering something not correct. If you don't know the answer to a question, please don't share false information.
""".strip()


def generate_prompt(prompt: str, system_prompt: str = DEFAULT_SYSTEM_PROMPT) -> str:
    return f"""
[INST] <<SYS>>
{system_prompt}
<</SYS>>
{prompt} [/INST]
""".strip()

In [ ]:
SYSTEM_PROMPT = "Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer."

template = generate_prompt(
    """
{context}

Question: {question}
""",
    system_prompt=SYSTEM_PROMPT,
)

In [ ]:
prompt = PromptTemplate(template=template, input_variables=["context", "question"])

# Pipline text-generation
## Edit output based on custom parameters
### Custom output use parameters:
 - max_new_tokens: tùy chỉnh ngữ cảnh đầu ra, nếu vượt quá, câu trả lời có thể bị cắt ngắn.
 - top_p: chi phối việc lựa chọn mã thông báo dựa trên xác suất.
 - temperature: xác định mức độ sáng tạo của câu trả lời. Giá trị thấp (gần bằng 0) tạo ra câu trả lời cụ thể hơn, trong khi giá trị cao (ví dụ: 1) tạo ra câu trả lời sáng tạo hơn.
 - repetition_penalty: chi phối việc lặp lại mã thông báo trong câu trả lời.


In [ ]:
streamer = TextStreamer(tokenizer, skip_prompt=True, skip_special_tokens=True)

text_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=3096,
    top_p=0.95,
    repetition_penalty=0.9,
    streamer=streamer,
)
llm = HuggingFacePipeline(pipeline=text_pipeline, model_kwargs={"temperature": 0.01})

# If you run it for the first time, run these blocks to load your data and embed it into the database, then you can use that database as a base knowledge.

In [ ]:
# docs = load_documents()
# text_chunks = split_text_into_chunks(docs)
# db = create_vector_store(text_chunks, embeddings)
# qa_chain = RetrievalQA.from_chain_type(
#   llm=llm,
#   chain_type="stuff",
#   retriever=db.as_retriever(search_kwargs={"k": 1}),
#   return_source_documents=True,
#   chain_type_kwargs={"prompt": prompt},
#   )

In [ ]:
# db.similarity_search("Current price of RHHBY", k=5)
# Results=qa_chain("Current price of RHHBY?")['result']


#Load ChormaDB
### load knowdegle base

In [ ]:
!rm -rf "/content/content"

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!unzip /content/drive/MyDrive/dbtrue.zip

In [ ]:
db1 = Chroma(persist_directory="/content/content/db", embedding_function=embeddings)
qa_chain1 = RetrievalQA.from_chain_type(
  llm=llm,
  chain_type="stuff",
  retriever=db1.as_retriever(search_kwargs={"k": 2}),
  return_source_documents=True,
  chain_type_kwargs={"prompt": prompt},
  )

In [ ]:
db1.similarity_search("Net income of AAPL stock last quarter", k=5)

In [ ]:
# db1.get(where={'source':'/content/drive/MyDrive/Data/financial_data (1).txt'})

In [ ]:
Results=qa_chain1("Revenue of AAPL in Q3 of 2022")['result']

In [ ]:
#!zip -r /content/drive/MyDrive/dbtrue.zip /content/db

# Chat with Multiple .txt file using gradio



In [ ]:
#Create interface use Gradio
import gradio as gr
import re

def is_greeting(message):
    greetings = ["hello", "hi", "hey", "greetings"]
    return any(greeting in message.lower() for greeting in greetings)

def predict(message, history):
    if is_greeting(message):
        return "Hello! I'm your StockScan.io ChatBot. How can I help you with stock-related queries today?"
    else:
        return qa_chain1(message)['result']

css_code_light = """
    .gradio-container {
        background-color: white;
        color: green;
    }
"""
demo = gr.ChatInterface(
    fn=predict,
    css=css_code_light,
    title='ChatBot US_Stock StockScan.io',
    theme=theme,
    input_size=(400, 50)
    examples=[
        ["Revenue of AAPL in Q3 of 2022"],
        ["In the Q1 of 2022, Revenue of AAPL stock"],
        ["Revenue of AAPL stock in the last quarter"],
        ["Revenue of AAPL stock now"],
        ["Current price of TESLA"],
    ],
    )
demo.launch(share=True)

# Update data real time and run gradio



In [ ]:
import json
with open('/content/drive/MyDrive/demo_websocket/list_stock.json', 'r') as f:
      data_stocks = json.load(f)
symbols = []  # Add more symbols if needed

for i in range(0,len(data_stocks['results'])):
  symbols.append(data_stocks['results'][i]['symbol'])

In [ ]:
import socketio
import json
import threading
import gradio as gr
# Init Socket.IO client
sio = socketio.Client(logger=True, engineio_logger=True)

# list stock
listcoins = symbols[:50]

# price of stock
coin_prices = {}

# Lock to ensure thread-safe access to shared data
lock = threading.Lock()

@sio.on('connect')
def on_connect():
    print("Connected")
    sio.emit("RealTimeAvgPriceSubAdd", {
        'subs': listcoins
    })

@sio.on('avg_price_update')
def handle_global_price_update(data):
   with lock:
        try:
            stock = data['symbol']
            price = data['price']
            coin_prices[stock] = price

            # if all(symbol in coin_prices for symbol in listcoins):
            with open('/content/drive/MyDrive/data/prices.txt', 'w') as txt_file:
              for symbol, price in coin_prices.items():
                txt_file.write(f"Current price of {symbol} stock is {price}$\n")
        except KeyError as e:
            print(f"Error: Missing key {e} in data.")

@sio.on('disconnect')
def disconnect():
    print('Disconnected')
sio.connect(url='your web socket url', transports=['websocket'])
def is_greeting(message):
    greetings = ["hello", "hi", "hey", "greetings"]
    return any(greeting in message.lower() for greeting in greetings)

def predict(message, history):
    if is_greeting(message):
        return "Hello! I'm your StockScan.io ChatBot. How can I help you with stock-related queries today?"
    else:
        return qa_chain1(message)['result']
demo = gr.ChatInterface(
    fn=predict,
    title='ChatBot US_Stock StockScan.io',
    examples=[
        ["Revenue of AAPL in Q3 of 2022"],
        ["In the Q1 of 2022, Revenue of AAPL stock"],
        ["Revenue of AAPL stock in the last quarter"],
        ["Revenue of AAPL stock now"],
        ["Current price of TSLA"],
    ],
)

demo.launch(share=True)
while(True):
    loader = DirectoryLoader('/content/drive/MyDrive/data', glob="*.txt", loader_cls=TextLoader)
    documents1 = loader.load()
    text_chunks1 = split_text_into_chunks(documents1)
    list_id=db1.get(offset=906227,limit=50)['ids']
    if list_id:
      for i in range(len(text_chunks1)):
          db1.update_document(document_id=list_id[i], document=text_chunks1[i])

sio.wait()
